In [ ]:
# ==========================================
# 🏗️ Notebook 2: Concrete Crack Detection (CNN)
# ==========================================

# 1. Install Dependencies
!pip install kagglehub tensorflow matplotlib

import kagglehub
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import os

# --- 2. Data Loading ---
print("Downloading Crack Images Dataset...")
# If this fails, ensure you have internet access or download manually
path = kagglehub.dataset_download("arnavr10880/concrete-crack-images-for-classification")

BATCH_SIZE = 32
IMG_SIZE = (128, 128)

print(f"Loading images from: {path}")
train_ds = tf.keras.utils.image_dataset_from_directory(
    path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='binary'
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='binary'
)

# Optimization
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# --- 3. Model Architecture (CNN) ---
model_cnn = models.Sequential([
    layers.Rescaling(1./255, input_shape=(128, 128, 3)),
    
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model_cnn.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# --- 4. Training ---
print("Training Crack Detection Model...")
history = model_cnn.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)

# --- 5. Evaluation ---
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.show()

# --- 6. Save Model for App ---
model_cnn.save('crack_detection_model.keras')
print("✅ Model saved to 'crack_detection_model.keras'")

100%|███████████████████████████████████████████████████████████████████████████████| 233M/233M [01:04<00:00, 3.79MB/s]

Extracting files...


Loading images from: C:\Users\adity\.cache\kagglehub\datasets\arnavr10880\concrete-crack-images-for-classification\versions\1
Found 40000 files belonging to 2 classes.
Using 32000 files for training.
Found 40000 files belonging to 2 classes.
Using 8000 files for validation.
Training Crack Detection Model...
Epoch 1/5


C:\Users\adity\anaconda3\Lib\site-packages\keras\src\layers\preprocessing\data_layer.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


 586/1000 ━━━━━━━━━━━━━━━━━━━━ 57s 140ms/step - accuracy: 0.8368 - loss: 0.3545 